## Summary: how to build a covariance matrix

We consider `INPUT` = `FITOPT000_MUOPT000.FITRES`. Based on this choice, we can calculate the different contributions to the covariance matrix. The full covariance `_COVd.txt` is then given by $$C = C_{fit} + C_{stat} + C_{dupl} + C_{FITOPTS} + C_{MUOPTS}$$

### SALT2 fit covariance $C_{fit}$

1. from each line in `INPUT` (which corresponds to an observation of a supernova event, not necessarily unique events) build a blockdiagonal matrix from `COV_x0_x1` etc variables: $$\sigma^2_{SNe, survey} = \left(\begin{array}{ccc} \Delta x_0^2 & \sigma_{x_0, x_1} & \sigma_{x_0, c} \\ \sigma_{x_0, x_1} & \Delta x_1^2 & \sigma_{c, x_1} \\ \sigma_{x_0, c} & \sigma_{c, x_1} & \Delta c^2\end{array}\right)$$
2. transform this matrix to the $(m_B, x_1, c)$ space:
$$\mathbb{Cov}(m_B, x_1, c) = J \cdot \mathbb{Cov}(x_0, x_1, c) \cdot J^T$$  using $m_B = -2.5 \cdot \log_{10}x_0 + c$ and thus $J = \left(\begin{array}{ccc}
\frac{-2.5}{x_0\ln 10} & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1\\
\end{array}\right)$
3. construct a block diagonal $3N \times 3N$ matrix from the $3 \times 3$ $\sigma_{SNe}^2$ blocks for each SNe $$C_{SALT2} = \left(\begin{array}{ccc}  f_1 \cdot \sigma_{SNe, 1}^2 & & 0 \\ & \ddots & \\ 0 & & f_N \cdot \sigma_{SNe, N}^2\end{array}\right)$$ where in Pantheon+ $f$ is a survey-dependent scaling to account for selection effects (see below) which we set to be $f=1$.
4. For some SNe, the $\sigma_{SNe}^2$ block matrix is not positive semi-definit. In these cases, the respective SNe is dropped from the final covariance $C$ as well as from the `INPUT` file.

### statistical covariance $C_{stat}$

Brout et al. 2022 (https://iopscience.iop.org/article/10.3847/1538-4357/ac8e04/pdf) consider a statistical covariance $C_{stat}$ calculated by $$C_{stat} = \begin{cases} f \cdot \sigma_{meas}^2 + &\sigma_{scat}^2 + \sigma_{gray}^2 + \sigma_{lens}^2 + \sigma_z^2 + \sigma_{vpec}^2 & \text{same SNe and same survey} \\ &\sigma_{scat}^2 + \sigma_{gray}^2 + \sigma_{lens}^2 + \sigma_z^2 + \sigma_{vpec}^2 & \text{same SNe, different surveys} \\ 0 & & \text{other off-diagonal entries}\end{cases}$$
(Equations (3), (4) and (5)). The different terms are given as follows:
- $f$ is a survey-dependent scaling to account for selection effects
- $\sigma_{meas}^2$ gives measurement uncertainty of SALT2 light-curve fit parameters
- $\sigma_{scat}^2$ describes modelled intrinsic brightness fluctuations
- $\sigma_{gray}^2$ is constant (added to $\sigma_{scat}^2$ to give $\sigma_{floor}^2$), determined after the BBC fitting process
- $\sigma_{lens} = 0.055 z$
- $\sigma_z^2$ as given in the `.FITRES` files 
- $\sigma_{vpec}^2$ as given in the `.FITRES` files   
  
In our case, the measurement uncertainty term $f \cdot \sigma_{meas}^2$ is given by $C_{fit}$. $\sigma_{vpec}^2$ is not relevant as we are not considering any peculiar velocities.  
The detailed treatment of $f, \sigma_{scat}^2, \sigma_{gray}^2$ in Brout et al. 2022 is performed under specific cosmological assumptions as they are infered from the bias correction. This is why we don't consider them in our analysis but adopt $f=1, \sigma_{floor}^2 = \sigma_{scat}^2 + \sigma_{gray}^2 = 0$.
From the uncertainties accociated with lensing and redshift, we obtain $$C_{stat}^{ij} = \left(\begin{array}{ccc}
\sigma_{lens}^2 + \sigma_{z}^2 & 0 & 0\\ 0 & 0 & 0\\ 0 & 0 & 0
\end{array}\right)$$
for each survey combination $i, j$ and each SNe. The combination of these then gives
$$C_{stat} = \left(\begin{array}{ccc} \begin{array}{ccc}
\sigma_{lens, 1}^2 + \sigma_{z, 1}^2 & 0 & 0\\ 0 & 0 & 0\\ 0 & 0 & 0\\
\end{array} & & 0 \\ & \ddots & \\ 0 & & \begin{array}{ccc}
\sigma_{lens, N}^2 + \sigma_{z, N}^2 & 0 & 0\\ 0 & 0 & 0\\ 0 & 0 & 0\\
\end{array}\end{array}\right)$$ as a (block) diagonal $3N \times 3N$ matrix to combine with the other covariances.

### duplicated SNe $C_{dupl}$

For each SNe, construct a matrix $C_{dupl, SNe} = \delta_{SNe} \cdot \delta_{SNe}^T$ where
$\delta_{SNe} = \left(\begin{array}{c} \vdots\\ m_{B, j}\\ x_{1, j}\\ c_j\\ \vdots\end{array}\right)_{SNe} - \left(\begin{array}{c} \vdots\\ \overline{m_{B}}\\ \overline{{x}_{1}}\\ \overline{c}\\ \vdots\end{array}\right)_{SNe}$ with $j$ running over all surveys. For a given SNe, the $\overline{m_{B}}, \overline{{x}_{1}}, \overline{c}$ have the same values for all surveys. Additionally, $0.102mag$ is added to all off-diagonal $(m_B, m_B)$ components, as this is the standard deviation of the differences of duplicated SNe according to Scolnic et al. 2022 (https://arxiv.org/pdf/2112.03863.pdf). Again, all blocks for individual SNe (of size $3\nu \times 3 \nu$ for $\nu$ surveys) are combined to a block-diagonal $3N \times 3N$ matrix:
$$C_{dupl} = \left(\begin{array}{ccc}  \sigma_{SNe, 1}^2 & & 0 \\ & \ddots & \\ 0 & & \sigma_{SNe, N}^2\end{array}\right)$$

### systematic covariance from `FITOPTS` $C_{FITOPTS}$

Equation (7) from Brout et al. 2022 (https://iopscience.iop.org/article/10.3847/1538-4357/ac8e04/pdf) states $$C^{ij}_{syst} = \sum\limits_\psi \frac{\partial \Delta \mu^i_\psi}{\partial S_\psi} \frac{\partial \Delta \mu^j_\psi}{\partial S_\psi} \sigma_\psi^2$$ which translates to  
`diff = scale * ((df1[VARNAME_MU] - df1[VARNAME_MUREF]) - \
                    (df2[VARNAME_MU] - df2[VARNAME_MUREF])).to_numpy()
    diff[~np.isfinite(diff)] = 0
    cov = diff[:, None] @ diff[None, :]`  
in the `create_covariance.py` file (function `get_cov_from_diff(...)`) in the SNANA GitHub repo to get the individual $\psi$ terms that are summed up later.  
In our case we choose `df2`=`INPUT` and vary `df1` across the other `FITOPT` files. We assume `df1[VARNAME_MUREF]`=`df2[VARNAME_MU]` which reduces `diff` to `scale * (df1[VARNAME_MU]- df2[VARNAME_MU])`. Furthermore, we want to consider $m_B, x_1, c$ instead of $\mu$ only. Thus, our calculation is as follows:
$$C_{FITOPTS} = \sum\limits_\psi \delta_\psi \cdot \delta_\psi^T$$
where $\psi$ varies across the different `FITOPT` files. $\delta_\psi$ is defined as $\delta_\psi = \left(\left(\begin{array}{c} \vdots\\m_{B, i}\\ x_{1, i}\\ c_i\\ \vdots\end{array}\right)_{\psi} - \left(\begin{array}{c} \vdots\\m_{B, i}\\ x_{1, i}\\ c_i\\ \vdots\end{array}\right)_{INPUT}\right) \cdot \sigma_\psi$ with $i$ enumerating the SNe in `INPUT`.

### systematic covariance from `MUOPTS` $C_{MUOPTS}$
The systematic covariance given by the `MUOPT` files can be calculated from the same formulae as $C_{FITOPTS}$ by simply varying $\psi$ across the `MUOPT` files instead of the `FITOPTS`.